Підготовка даних

In [ ]:
import pandas as pd

# Завантаження даних
data = pd.read_csv('AirQuality.csv')

# Перегляд доступних стовпців
print(data.columns)
print(data)


Index(['Date;Time;CO(GT);PT08.S1(CO);NMHC(GT);C6H6(GT);PT08.S2(NMHC);NOx(GT);PT08.S3(NOx);NO2(GT);PT08.S4(NO2);PT08.S5(O3);T;RH;AH;;'], dtype='object')
                                                                                                            Date;Time;CO(GT);PT08.S1(CO);NMHC(GT);C6H6(GT);PT08.S2(NMHC);NOx(GT);PT08.S3(NOx);NO2(GT);PT08.S4(NO2);PT08.S5(O3);T;RH;AH;;
10/03/2004;18.00.00;2            6;1360;150;11                 9;1046;166;1056;113;1692;1268;13 6;48 9;0                                                7578;;                                                                          
10/03/2004;19.00.00;2;1292;112;9 4;955;103;1174;92;1559;972;13 3;47                             7;0  7255;;                                                NaN                                                                          
10/03/2004;20.00.00;2            2;1402;88;9                   0;939;131;1140;114;1555;1074;11  9;54 0;0                                             

Тренування моделі

In [ ]:
import pandas as pd
import numpy as np



# Завантаження даних з правильним роздільником
data = pd.read_csv('AirQuality.csv', sep=';')

# Перевірка доступних стовпців у датафреймі
print(data.columns)

# Перегляд перших декількох рядків для зразка даних
print(data.head())

# Вибір конкретного стовпця (наприклад, 'NO2(GT)')
data['NO2(GT)'].fillna(method='bfill', inplace=True)  # Заповнення пропусків для обраного стовпця

# Нормалізація вибраного стовпця
data_scaled = (data['NO2(GT)'] - data['NO2(GT)'].mean()) / data['NO2(GT)'].std()

# Створення пакетів даних
window_size = 24  # використовуємо дані за останні 24 години для прогнозу
X, y = [], []
for i in range(window_size, len(data_scaled)):
    X.append(data_scaled[i-window_size:i])
    y.append(data_scaled[i])

# Поділ даних на навчальну і валідаційну вибірки
X_train, X_val = np.array(X[:int(len(X)*0.8)]), np.array(X[int(len(X)*0.8):])
y_train, y_val = np.array(y[:int(len(y)*0.8)]), np.array(y[int(len(y)*0.8):])


Index(['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)', 'T', 'RH', 'AH', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')
         Date      Time CO(GT)  PT08.S1(CO)  NMHC(GT) C6H6(GT)  PT08.S2(NMHC)  \
0  10/03/2004  18.00.00    2,6       1360.0     150.0     11,9         1046.0   
1  10/03/2004  19.00.00      2       1292.0     112.0      9,4          955.0   
2  10/03/2004  20.00.00    2,2       1402.0      88.0      9,0          939.0   
3  10/03/2004  21.00.00    2,2       1376.0      80.0      9,2          948.0   
4  10/03/2004  22.00.00    1,6       1272.0      51.0      6,5          836.0   

   NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)     T    RH  \
0    166.0        1056.0    113.0        1692.0       1268.0  13,6  48,9   
1    103.0        1174.0     92.0        1559.0        972.0  13,3  47,7   
2    131.0        1140.0    114.0        15

Згорткова мережа CNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten


num_features = 1  # тому що ми використовуємо один стовпець даних

# Ініціалізація моделі згорткової мережі
model_cnn = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(window_size, num_features)),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

# Компіляція моделі
model_cnn.compile(optimizer='adam', loss='mse')

# Навчання моделі
model_cnn.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
237/237 [==============================] - 2s 5ms/step - loss: 0.1887 - val_loss: nan
Epoch 2/10
237/237 [==============================] - 1s 4ms/step - loss: 0.1067 - val_loss: nan
Epoch 3/10
237/237 [==============================] - 1s 4ms/step - loss: 0.0969 - val_loss: nan
Epoch 4/10
237/237 [==============================] - 1s 5ms/step - loss: 0.0862 - val_loss: nan
Epoch 5/10
237/237 [==============================] - 1s 4ms/step - loss: 0.0869 - val_loss: nan
Epoch 6/10
237/237 [==============================] - 2s 10ms/step - loss: 0.0830 - val_loss: nan
Epoch 7/10
237/237 [==============================] - 1s 5ms/step - loss: 0.0807 - val_loss: nan
Epoch 8/10
237/237 [==============================] - 1s 4ms/step - loss: 0.0783 - val_loss: nan
Epoch 9/10
237/237 [==============================] - 1s 4ms/step - loss: 0.0786 - val_loss: nan
Epoch 10/10
237/237 [==============================] - 1s 4ms/step - loss: 0.0770 - val_loss: nan


Трансформер

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout

# Параметри моделі
num_features = 1  # тому що ми використовуємо один стовпець даних
window_size = 24  # використовуємо дані за останні 24 години для прогнозу

# Функція для створення одного трансформер-блоку
def transformer_block(inputs):
    # Самоувага
    attn_output = MultiHeadAttention(num_heads=2, key_dim=2)(inputs, inputs)
    attn_output = Dropout(0.1)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)

    # Повнозв'язний шар
    ffn_output = Dense(64, activation="relu")(out1)
    ffn_output = Dropout(0.1)(ffn_output)
    return LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

# Вхідні дані
inputs = Input(shape=(window_size, num_features))

# Трансформер-блок
x = transformer_block(inputs)

# Вихідний шар
outputs = Dense(1)(x)

# Створення моделі
model_transformer = Model(inputs=inputs, outputs=outputs)

# Компіляція моделі
model_transformer.compile(optimizer='adam', loss='mse')

# Вивід структури моделі
model_transformer.summary()

# Навчання моделі
model_transformer.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 24, 1)]              0         []                            
                                                                                                  
 multi_head_attention_6 (Mu  (None, 24, 1)                29        ['input_2[0][0]',             
 ltiHeadAttention)                                                   'input_2[0][0]']             
                                                                                                  
 dropout_10 (Dropout)        (None, 24, 1)                0         ['multi_head_attention_6[0][0]
                                                                    ']                            
                                                                                            

2.Ознайомитись з наведеними прикладом використання трансформерів для мовних моделей

In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
txt_1 = 'Football is the most exciting and popular game in the world'
classifier(txt_1)
txt_2 = 'The weather is unpredictable around here'
classifier(txt_2)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.6950812935829163}]